# Parallelization Lab

In this lab, you will be leveraging several concepts you have learned to obtain a list of links from a web page and crawl and index the pages referenced by those links - both sequentially and in parallel. Follow the steps below to complete the lab.

## Step 1: Use the requests library to retrieve the content from the URL below.

In [ ]:
import requests
url = 'https://en.wikipedia.org/wiki/Data_science'


In [ ]:
req = requests.get(url)

## Step 2: Use BeautifulSoup to extract a list of all the unique links on the page.

In [ ]:
from bs4 import BeautifulSoup

In [ ]:
soup = BeautifulSoup(req.content,'html.parser')
soup.find_all(['link','a'])[0]

In [ ]:
links = []
for link in soup.find_all( href = True):
    if link['href'][0]=='h' or link['href'][0]=='/':
        links.append(link['href'])
links

## Step 3: Use list comprehensions with conditions to clean the link list.

There are two types of links, absolute and relative. Absolute links have the full URL and begin with http while relative links begin with a forward slash (/) and point to an internal page within the wikipedia.org domain. Clean the respective types of URLs as follows.

- Absolute Links: Create a list of these and remove any that contain a percentage sign (%).
- Relativel Links: Create a list of these, add the domain to the link so that you have the full URL, and remove any that contain a percentage sign (%).
- Combine the list of absolute and relative links and ensure there are no duplicates.

In [ ]:
domain = 'http://wikipedia.org'

In [ ]:
link_abs = []
link_rel = []
for l in links:
    if '%' in l:
        pass
    elif 'http' in l:
        link_abs.append(l)    
    elif l[0]=='/' and l[1]=='/':
        link_rel.append('http:'+l)
    elif l[0]=='/':
        link_rel.append(domain+l)
        

links_http = list(set(link_rel +link_abs))
links_http

## Step 4: Use the os library to create a folder called *wikipedia* and make that the current working directory.

In [ ]:
import os
os.getcwd()
os.chdir('/Users/saulsoni/datamex_102020/module-1/lab-parallelization/your-code')

In [ ]:
try:
    os.mkdir('wikipedia')
    print("Directory Created ") 
except FileExistsError:
    print("Directory already exists")
os.chdir('wikipedia')
os.getcwd()

## Step 5: Write a function called index_page that accepts a link and does the following.

- Tries to request the content of the page referenced by that link.
- Slugifies the filename using the `slugify` function from the [python-slugify](https://pypi.org/project/python-slugify/) library and adds a .html file extension.
    - If you don't already have the python-slugify library installed, you can pip install it as follows: `$ pip install python-slugify`.
    - To import the slugify function, you would do the following: `from slugify import slugify`.
    - You can then slugify a link as follows `slugify(link)`.
- Creates a file in the wikipedia folder using the slugified filename and writes the contents of the page to the file.
- If an exception occurs during the process above, just `pass`.

In [ ]:
from slugify import slugify
slugify(links_http[0])

In [ ]:
def index_page(link):
    try:
        soup = BeautifulSoup(requests.get(link).content, 'html.parser')
        
        file = open(f'{slugify(link)}.html', "x", encoding = 'UTF-8')
        file.write(soup.prettify())
        file.close()
    except:
        print('Couldnt Load')
index_page(links_http[0])

## Step 6: Sequentially loop through the list of links, running the index_page function each time.

Remember to include `%%time` at the beginning of the cell so that it measures the time it takes for the cell to run.

In [ ]:
%%time

for link in links_http:
    index_page(link)


## Step 7: Perform the page indexing in parallel and note the difference in performance.

Remember to include `%%time` at the beginning of the cell so that it measures the time it takes for the cell to run.

In [ ]:
from multiprocessing import Pool


In [ ]:
%%time

pool = Pool(4) 
result = pool.map(index_page, links_http)
pool.terminate()
pool.join()        
result